In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 23 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

29 March 2023 ; 12:25:13


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,32,0,9.4,1.5,3.4,0.440,...,4.36,115.92,111.72,4.20,4.14,115.78,111.79,4.00,1,1
12,MarJon Beauchamp,SF,22,MIL,45,9,13.3,1.9,4.8,0.402,...,4.36,115.92,111.72,4.20,4.14,115.78,111.79,4.00,1,1
18,A.J. Lawson,SG,22,DAL,13,0,4.8,1.0,2.0,0.500,...,0.57,117.59,117.05,0.53,0.35,117.53,117.22,0.31,1,1
26,Jaden Hardy,SG,20,DAL,42,3,14.6,3.0,6.8,0.439,...,0.57,117.59,117.05,0.53,0.35,117.53,117.22,0.31,1,1
39,AJ Griffin,SF,19,ATL,67,11,19.3,3.4,7.4,0.467,...,0.18,117.31,117.11,0.20,0.08,117.49,117.40,0.09,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.8,ORL,14.8,0.356522,2.614493,6.7,19.9,13,0.244176
1,Walker Kessler,22.9,UTA,21.6,2.309859,7.853521,4.0,9.1,12,0.199049
2,Jaden Ivey,30.6,DET,12.0,-0.734328,0.000000,5.3,15.7,15,0.087109
3,Jalen Duren,24.6,DET,17.4,0.683333,5.603333,3.7,8.9,15,0.072421
4,Jalen Williams,30.2,OKC,15.7,1.522857,6.208571,5.4,13.9,10,0.071232


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)